# Przetwarzanie surowych danych

## Import bibliotek

In [1]:
import pandas as pd
import datetime

## Wczytanie danych

### Dane dotyczące zużycia energii

In [2]:
energy_data = pd.read_csv(
    '../data/raw/energy_dataset.csv', 
    usecols=['time', 'total load actual']
)

In [3]:
energy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time               35064 non-null  object 
 1   total load actual  35028 non-null  float64
dtypes: float64(1), object(1)
memory usage: 548.0+ KB


In [4]:
energy_data.head()

,time,total load actual
0,2015-01-01 00:00:00+01:00,25385.0
1,2015-01-01 01:00:00+01:00,24382.0
2,2015-01-01 02:00:00+01:00,22734.0
3,2015-01-01 03:00:00+01:00,21286.0
4,2015-01-01 04:00:00+01:00,20264.0


### Dane pogodowe

In [5]:
weather_data = pd.read_csv(
    '../data/raw/weather_features.csv',
    usecols=['dt_iso', 'city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'rain_1h', 'clouds_all']
)

In [6]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178396 entries, 0 to 178395
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   dt_iso      178396 non-null  object 
 1   city_name   178396 non-null  object 
 2   temp        178396 non-null  float64
 3   pressure    178396 non-null  int64  
 4   humidity    178396 non-null  int64  
 5   wind_speed  178396 non-null  int64  
 6   rain_1h     178396 non-null  float64
 7   clouds_all  178396 non-null  int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 10.9+ MB


In [7]:
weather_data.head()

,dt_iso,city_name,temp,pressure,humidity,wind_speed,rain_1h,clouds_all
0,2015-01-01 00:00:00+01:00,Valencia,270.475,1001,77,1,0.0,0
1,2015-01-01 01:00:00+01:00,Valencia,270.475,1001,77,1,0.0,0
2,2015-01-01 02:00:00+01:00,Valencia,269.686,1002,78,0,0.0,0
3,2015-01-01 03:00:00+01:00,Valencia,269.686,1002,78,0,0.0,0
4,2015-01-01 04:00:00+01:00,Valencia,269.686,1002,78,0,0.0,0


## Czyszczenie i przygotowanie danych

### Scalenie tabel

In [8]:
# Parsowanie czasu
weather_data['dt_iso'] = pd.to_datetime(weather_data['dt_iso'], utc=True)
energy_data['time'] = pd.to_datetime(energy_data['time'], utc=True)

In [9]:
weather_data['city_name'].unique()

array(['Valencia', 'Madrid', 'Bilbao', ' Barcelona', 'Seville'],
      dtype=object)

Zużycie energii dotyczy całej Hiszpanii, zatem dane z 5 największych miast zostaną uśrednione, aby otrzymać pogląd całościowy.

In [10]:
# Grupowanie po czasie – średnia z 5 miast na każdą godzinę
weather_avg = (
    weather_data
    .drop('city_name', axis=1)
    .groupby(['dt_iso'])
    .mean()
    .reset_index()
    .rename(columns={'dt_iso': 'time'})
)

In [11]:
# Zmiana nazwy kolumny w energy_data
energy_data.rename(columns={'total load actual': 'total_load'}, inplace=True)

In [12]:
# Scalanie
df_merged = pd.merge(energy_data, weather_avg, on='time', how='inner')
df_merged.head()

,time,total_load,temp,pressure,humidity,wind_speed,rain_1h,clouds_all
0,2014-12-31 23:00:00+00:00,25385.0,272.491463,1016.4,82.4,2.0,0.0,0.0
1,2015-01-01 00:00:00+00:00,24382.0,272.512700,1016.2,82.4,2.0,0.0,0.0
2,2015-01-01 01:00:00+00:00,22734.0,272.099137,1016.8,82.0,2.4,0.0,0.0
3,2015-01-01 02:00:00+00:00,21286.0,272.089469,1016.6,82.0,2.4,0.0,0.0
4,2015-01-01 03:00:00+00:00,20264.0,272.145900,1016.6,82.0,2.4,0.0,0.0


### Brakujące wartości

In [13]:
df_merged.isnull().sum()

time           0
total_load    36
temp           0
pressure       0
humidity       0
wind_speed     0
rain_1h        0
clouds_all     0
dtype: int64

In [14]:
df_merged[df_merged['total_load'].isnull()].head(2)

,time,total_load,temp,pressure,humidity,wind_speed,rain_1h,clouds_all
108,2015-01-05 11:00:00+00:00,NaN,283.588731,1012.6,76.4,1.2,0.00,13.0
109,2015-01-05 12:00:00+00:00,NaN,284.470200,1012.2,74.2,2.0,0.06,21.2


In [15]:
# Uzupełnienie brakujących wartości przy pomocy interpolacji czasowej na podstawie sąsiednich wierszy
df_merged = df_merged.set_index('time')
df_merged['total_load'] = df_merged['total_load'].interpolate(method='time')
df_merged = df_merged.reset_index()

df_merged.isnull().sum()

time          0
total_load    0
temp          0
pressure      0
humidity      0
wind_speed    0
rain_1h       0
clouds_all    0
dtype: int64

### Przygotowanie nowej tablicy

In [16]:
# Sprawdzenie, czy dla danego dnia są wiersze dla każdej godziny
df_merged.groupby(df_merged['time'].dt.date)['time'].count().unique()

array([ 1, 24, 23])

In [17]:
# Szukanie niepełnych dni
display(df_merged.head(2))
display(df_merged.tail(2))

,time,total_load,temp,pressure,humidity,wind_speed,rain_1h,clouds_all
0,2014-12-31 23:00:00+00:00,25385.0,272.491463,1016.4,82.4,2.0,0.0,0.0
1,2015-01-01 00:00:00+00:00,24382.0,272.512700,1016.2,82.4,2.0,0.0,0.0


,time,total_load,temp,pressure,humidity,wind_speed,rain_1h,clouds_all
35062,2018-12-31 21:00:00+00:00,25801.0,279.718,1030.2,70.0,2.8,0.0,0.0
35063,2018-12-31 22:00:00+00:00,24455.0,278.798,1030.2,82.4,2.6,0.0,0.0


In [18]:
# Usunięcie wiersza z rokiem 2014
df_merged = df_merged.iloc[1:].reset_index(drop=True)

# Sztuczne dodanie godziny 23 do wiersza z 2018-12-31
last_2018_row = df_merged[df_merged['time'].dt.year == 2018].iloc[-1].copy()
last_time = last_2018_row['time']
last_time = last_time.replace(hour=23)
last_2018_row['time'] = last_time

# Dodaj nowy wiersz do DataFrame
df_merged = pd.concat([df_merged, pd.DataFrame([last_2018_row])], ignore_index=True)

In [19]:
display(df_merged.head(2))
display(df_merged.tail(2))

,time,total_load,temp,pressure,humidity,wind_speed,rain_1h,clouds_all
0,2015-01-01 00:00:00+00:00,24382.0,272.512700,1016.2,82.4,2.0,0.0,0.0
1,2015-01-01 01:00:00+00:00,22734.0,272.099137,1016.8,82.0,2.4,0.0,0.0


,time,total_load,temp,pressure,humidity,wind_speed,rain_1h,clouds_all
35062,2018-12-31 22:00:00+00:00,24455.0,278.798,1030.2,82.4,2.6,0.0,0.0
35063,2018-12-31 23:00:00+00:00,24455.0,278.798,1030.2,82.4,2.6,0.0,0.0


## Zapis przetworzonych danych

In [20]:
df_merged.to_csv('../data/processed/energy_and_weather_data.csv', index=False)